# Data export and classification

In [1]:
import os
import numpy as np
import qcodes as qc
import nanotune as nt

from nanotune.data.export_data import correct_normalizations, export_data, prep_data
from nanotune.classification.classifier import Classifier
nt_root = os.path.dirname(os.path.dirname(os.path.abspath(nt.__file__)))

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/jana/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/jana/.qcodes/logs/210810-22969-qcodes.log


## Export labelled data to a numpy file

In [2]:
export_data('pinchoff', ['device_characterization.db'], db_folder=os.path.join(nt_root, 'data', 'tuning'), filename='test_export_data')

## Train classifier with the data saved in the numpy file. 
Note that this is just to small demo and that the data in the file is not sufficient for real-life purposes.

In [3]:
pinchoff_classifier = Classifier(
        ['test_export_data.npy'],
        'pinchoff',
        data_types=["signal"],
        classifier="SVC",
        folder_path=os.path.join(nt_root, 'data', 'tuning'),
    )

In [4]:
pinchoff_classifier.train()

We can predict the quality of a dataset as follows. Note that this is again just to show how the code works and that a classifier should not be used to predict data it has trained on.

In [5]:
pinchoff_classifier.predict(1203, db_name='device_characterization.db', db_folder=os.path.join(nt_root, 'data', 'tuning'))

[array([1])]